In [ ]:
#This is the conjob command that will auto post images/videos to social networks
#format
#cert_cron certificate_id channel <media_type>
#example: cert_cron.py AWS_SAA LinkedIn
#PDF: 20 questions (Github)

In [20]:
SHORT_VIDEO_CHANNELS = ['Youtube', 'LinkedIn', 'Facebook', 'X', 'Tiktok']   #1 video = 3 images/questions
LONG_VIDEO_CHANNELS = ['Youtube']   #1 video = 20 images/questions
IMAGE_CHANNELS = ['PInterest', 'Reddit']    #1 image = 3 questions
PDF_CHANNELS = ['Github']   #1 PDF = 4 pages * 5 questions

In [2]:
#1 question should have flag to indicate which channel(s) it appeared. for ex: "fb": 20250528
#we should post to social network once per day

In [60]:
import sys
import os
import importlib
import pymongo
from dotenv import load_dotenv
import subprocess


# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Get the path to the parent directory
parent_dir = os.path.dirname(notebook_dir)

# Add the parent directory to sys.path if it's not already there
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [57]:
load_dotenv(override=True) 

import thirdparty_sdk
importlib.reload(thirdparty_sdk)    #always get latest code
import exam_settings
importlib.reload(exam_settings) #always get latest code
import const
importlib.reload(const) #always get latest code

<module 'const' from '/Users/sang/Documents/Source/Python/python_webscrap/const.py'>

In [13]:
db_client = pymongo.MongoClient(os.environ['DB_URI'])
db = db_client['db_certificates']

In [ ]:
def create_img_with_3_images(documents):
    exam_settings.generate_image_portrait_from_file('img_3_question_template.html', './', 'test_3_q.png')

#test
#create_img_with_3_images(None)

[5826:110457:0529/095532.495568:ERROR:net/cert/internal/trust_store_mac.cc:817] Error parsing certificate:
ERROR: Failed parsing extensions

[5832:110468:0529/095533.099789:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[5832:110468:0529/095533.101087:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[5832:110468:0529/095533.101712:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[5832:110468:0529/095533.303017:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[5832:110468:0529/095533.315528:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[5832:110468:0529/095533.374851:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[5832:110468:0529/095533.375857:ERROR:ui/gl/gl_display.cc:508] EGL Driver messa

In [26]:
def get_cert_metadata(cert_symbol):
    meta_collection = db['tb_cert_metadata']
    #get metadata of the certificate
    cert_metadata = meta_collection.find_one({'symbol': cert_symbol})
    return cert_metadata

In [27]:
def query_random_questions(num_of_questions, channel, cert_metadata):
    #query random questions
    condition = {'type': 'multiple-choice'}
    condition[channel] = None
    pipeline = [
                {"$match": condition},
                {"$sample": {"size": num_of_questions}} #randomly documents
            ]
    collection = db[cert_metadata['collection_name']]
    random_documents = list(collection.aggregate(pipeline))
    if len(random_documents) < num_of_questions:
        print('Not enough questions to export')
        return []
    return random_documents

In [62]:
def generate_1_img_multiple_questions(random_documents, cert_metadata, today_yyyymmdd):
    question_index = 1
    document_html = []
    document_html.append(exam_settings.html_head_1_img_3_q_str)
    #1. add header
    document_html.append('<div class="header">'+cert_metadata['name']+'</div>')
    #2. add questions
    for doc in random_documents:
        container_html = []
        container_html.append('<div class="container">')
        str_index = str(question_index) + ') '
        #question first
        container_html.append('<div class="question">'+str_index + doc['question']+'</div>')
        #2.1 add answers
        container_html.append('<div class="answers">')
        for key in doc['options'].keys():
            container_html.append( f'''
                    <div class="answer">
                        <label>{key}. {doc['options'][key]}</label>
                    </div>''')
        container_html.append('</div>') #end list of answers
        container_html.append('</div>') #end 1 container
        document_html.append(''.join(container_html))
        question_index += 1
    #add footer (optional because image/video cannot click on the link)
    # document_html.append('<div class="footer">Check out more questions via <a href="'+cert_metadata['udemy_link']+'">this link</a></div>')
    #end doc
    document_html.append(exam_settings.html_tail_1_img_3_q_str)
    #1 doc 1 image
    filename = 'img_multi_q_'+today_yyyymmdd + '.png'
    creation_result = exam_settings.generate_image_portrait(''.join(document_html), cert_metadata['img_m_q_folder_path'], filename)
    return creation_result, filename

In [63]:
#create video with only 1 image
#sample: ffmpeg -loop 1 -i input_image.jpg -i input_audio.mp3 -c:v libx264 -tune stillimage -c:a aac -b:a 192k -pix_fmt yuv420p -t 10 -shortest output_video.mp4
def create_video_from_image(img_path, output_filename):
    audio_path = os.path.abspath('audio/bg_audio_16sec.m4a')
    abs_output_path = os.path.abspath(output_filename)
    ffmpeg_command = [
        "/Users/sang/Downloads/SetupFiles/ffmpeg/ffmpeg",
        "-loop", "1",
        "-i", img_path,
        "-i", audio_path,
        "-c:v", "libx264",
        "-tune", "stillimage",
        "-c:a", "aac",
        "-b:a", "192k",
        "-pix_fmt", "yuv420p",
        "-t", "10",
        "-shortest",
        "-y", # Overwrite output file without asking
        abs_output_path
    ]

    try:
        result = subprocess.run(
            ffmpeg_command,
            # cwd=abs_image_folder, # This changes the directory for THIS command execution
            check=True,
            capture_output=True,
            text=True
        )
        print(f"Video '{output_filename}' created successfully at {abs_output_path}")
        return True
    except FileNotFoundError:
        print(f"Error: FFmpeg command not found. Make sure FFmpeg is installed and in your system's PATH.")
        return False
    except subprocess.CalledProcessError as e:
        print(f"Error: FFmpeg command failed with exit code {e.returncode}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [64]:
def main(cert_symbol, channel):
    num_of_questions = 3    #default we export 3 questions
    if channel in LONG_VIDEO_CHANNELS or channel in PDF_CHANNELS:
        num_of_questions = 20
    #
    cert_metadata = get_cert_metadata(cert_symbol)
    #query random questions
    random_documents = query_random_questions(num_of_questions, channel, cert_metadata)
    if len(random_documents) == 0:
        return
    today_yyyymmdd = const.get_current_date_yyyymmdd()
    if channel in SHORT_VIDEO_CHANNELS or channel in IMAGE_CHANNELS:
        #create 1 image with multiple questions
        creation_result, filename = generate_1_img_multiple_questions(random_documents, cert_metadata, today_yyyymmdd)
        if creation_result == False:
            return
        if channel in SHORT_VIDEO_CHANNELS:
            #create video with 1 image only
            create_video_from_image(cert_metadata['img_m_q_folder_path']+filename, cert_metadata['img_m_q_folder_path'] + 'vid_multi_q_'+today_yyyymmdd + '.mp4')

#test
if __name__ == '__main__':
    print(sys.argv)
    args = sys.argv
    if len(args) == 1:
        #todo there is no any param, get all certificates and post to all channels
        todo = 1

    # cert_symbol = args[1]   #certificate symbol
    # channel = args[2]
    #
    cert_symbol = 'AWS_SAA' #for testing
    channel = 'LinkedIn'
    #
    main(cert_symbol, channel)

['/Users/sang/.pyenv/versions/3.9.10/lib/python3.9/site-packages/ipykernel_launcher.py', '--f=/Users/sang/Library/Jupyter/runtime/kernel-v395174b41b0d3494fcc23a2b4da7097b8a4c11d95.json']


[10236:209589:0529/134855.995448:ERROR:net/cert/internal/trust_store_mac.cc:817] Error parsing certificate:
ERROR: Failed parsing extensions

[10242:209614:0529/134856.644134:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[10242:209614:0529/134856.747883:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[10242:209614:0529/134856.748744:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[10242:209614:0529/134856.806438:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[10242:209614:0529/134856.807419:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[10242:209614:0529/134856.854459:ERROR:ui/gl/gl_display.cc:508] EGL Driver message (Error) eglQueryDeviceAttribEXT: Bad attribute.
[10242:209614:0529/134856.856171:ERROR:ui/gl/gl_display.cc:508] EGL Driv

Video 'aws_sa_data/img_m_q/vid_multi_q_20250529.mp4' created successfully at /Users/sang/Documents/Source/Python/python_webscrap/cert_exam/aws_sa_data/img_m_q/vid_multi_q_20250529.mp4
